Primjer treniranja CNN-a na training setu velicine 261 i testiranje na balansiranom setu veličine 32

In [9]:

from __future__ import print_function
import numpy as np
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
import mfsc_cnn_fun as dataimport
from keras.models import model_from_json



max_features = 5000
maxlen = 100
batch_size = 32
embedding_dims = 100
nb_filter = 90
hidden_dims = 256
nb_epoch = 50
nb_classes = 2
optimizer=Adam(lr=0.0001,decay=1e-6)
loss = 'categorical_crossentropy'
model_json='hb_model_orthogonal_experiment_norm.json'
weights='Model1_weights.hdf5'
seed = 1995
test_split = 0.2
normal_path ='C:\\Users\\Josip\\test set\\normal\\'
abnormal_path ='C:\\Users\\Josip\\test set\\abnormal\\'


print('Loading data...')
#Loads training data from specified folders for normal and abnormal sound files. 
#Transfrorms data using short-time Fourier transform, logscales the result and splits it into 129x129 squares.
#Randomizes data.
#Splits the data into train and test arrays
(X_train, y_train), (X_test, y_test) = dataimport.load_data(normal_path=normal_path, abnormal_path=abnormal_path, test_split=test_split, width=129, height=256, seed=seed)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

print('Build model...')
model = Sequential()

# We start off with using Convolution2D for a frame
# The filter is 3x57
model.add(Convolution2D(nb_filter=nb_filter,
                        nb_row=3,
                        nb_col=57,
                        init='uniform',
                        border_mode='valid',
                        W_regularizer=l2(0.0001),
                        W_constraint = maxnorm(2),
                        input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),dim_ordering="th")) 
model.add(Activation('relu'))

# dropout to reduce overfitting:
model.add(Dropout(0.25))                 #### promijenio si na 0.

# we use standard max pooling (halving the output of the previous layer):
model.add(MaxPooling2D(pool_size=(3, 4), strides=(1, 3)))

# the second convolution layer is 1x3
model.add(Convolution2D(nb_filter, 
                        nb_row=1,
                        nb_col=3,
                        init='uniform', 
                        W_regularizer=l2(0.0001), 
                        W_constraint=maxnorm(2),dim_ordering='th'))   ###########dodao si dim_ordering='th'

model.add(Activation('relu'))
model.add(Dropout(0.25))

# we use max pooling again:
model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# we add two hidden layers:
# increasing number of hidden layers may increase the accuracy, current number is designed for the competition 
model.add(Dense(hidden_dims, 
                init='uniform', 
                W_regularizer=l2(0.0001), 
                W_constraint = maxnorm(2)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(hidden_dims, 
                init='uniform', 
                W_regularizer=l2(0.0001), 
                W_constraint = maxnorm(2)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

# We project onto a binary output layer to determine the category (Currently: normal/abnormal, but you can try train on the exact abnormality also)
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Compile the model
model.compile(loss=loss, optimizer=optimizer,metrics=['accuracy'])

# Saving model to Json (its easier to test it this way)    
json_string = model.to_json()
open(model_json, 'w').write(json_string)

# Each time the loss will drop it will save weights file
checkpointer = ModelCheckpoint(filepath=weights, verbose=1, save_best_only=True)

# Start training
history=model.fit(X_train, Y_train, batch_size=batch_size,
          nb_epoch=nb_epoch,
          shuffle=True,
          callbacks=[checkpointer],
          validation_data=(X_test, Y_test))


Loading data...


C:\Users\Josip\mfsc_cnn_fun.py:309: RuntimeWarning: divide by zero encountered in log10
  ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel


Normal frames test:296
Abnormal frames test:325
Normal frames validation:74
Abnormal frames validation:71


C:\Users\Josip\mfsc_cnn_fun.py:326: RuntimeWarning: divide by zero encountered in log10
  ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel


592 train sequences
142 test sequences
X_train shape: (592, 3, 129, 129)
X_test shape: (142, 3, 129, 129)
Y_train shape: (592, 2)
Y_test shape: (142, 2)
Build model...


C:\Users\Josip\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:65: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=90, padding="valid", input_shape=(3, 129, 1..., data_format="channels_first", kernel_size=(3, 57), kernel_initializer="uniform", kernel_regularizer=<keras.reg..., kernel_constraint=<keras.con...)`
C:\Users\Josip\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:80: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(90, (1, 3), kernel_regularizer=<keras.reg..., kernel_initializer="uniform", data_format="channels_first", kernel_constraint=<keras.con...)`
C:\Users\Josip\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_regularizer=<keras.reg..., kernel_initializer="uniform", kernel_constraint=<keras.con...)`
C:\Users\Josip\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:104: UserWarning:

Train on 592 samples, validate on 142 samples
Epoch 1/50
592/592 [==============================] - 6s 10ms/step - loss: 0.9151 - acc: 0.5659 - val_loss: 0.7452 - val_acc: 0.5000

Epoch 00001: val_loss improved from inf to 0.74521, saving model to Model1_weights.hdf5
Epoch 2/50
592/592 [==============================] - 4s 7ms/step - loss: 0.7481 - acc: 0.6233 - val_loss: 0.6838 - val_acc: 0.8028

Epoch 00002: val_loss improved from 0.74521 to 0.68382, saving model to Model1_weights.hdf5
Epoch 3/50
592/592 [==============================] - 4s 7ms/step - loss: 0.6919 - acc: 0.6453 - val_loss: 0.6942 - val_acc: 0.8239

Epoch 00003: val_loss did not improve from 0.68382
Epoch 4/50
592/592 [==============================] - 4s 7ms/step - loss: 0.6789 - acc: 0.6318 - val_loss: 0.6903 - val_acc: 0.8169

Epoch 00004: val_loss did not improve from 0.68382
Epoch 5/50
592/592 [==============================] - 4s 7ms/step - loss: 0.6794 - acc: 0.6486 - val_loss: 0.6886 - val_acc: 0.8239

Epoch 

Epoch 43/50
592/592 [==============================] - 4s 7ms/step - loss: 0.1358 - acc: 0.9645 - val_loss: 0.4845 - val_acc: 0.7394

Epoch 00043: val_loss did not improve from 0.40731
Epoch 44/50
592/592 [==============================] - 4s 7ms/step - loss: 0.1570 - acc: 0.9561 - val_loss: 0.4504 - val_acc: 0.7958

Epoch 00044: val_loss did not improve from 0.40731
Epoch 45/50
592/592 [==============================] - 4s 7ms/step - loss: 0.1247 - acc: 0.9865 - val_loss: 0.4089 - val_acc: 0.7958

Epoch 00045: val_loss did not improve from 0.40731
Epoch 46/50
592/592 [==============================] - 4s 7ms/step - loss: 0.1156 - acc: 0.9747 - val_loss: 0.4178 - val_acc: 0.7817

Epoch 00046: val_loss did not improve from 0.40731
Epoch 47/50
592/592 [==============================] - 4s 7ms/step - loss: 0.1146 - acc: 0.9780 - val_loss: 0.4710 - val_acc: 0.7254

Epoch 00047: val_loss did not improve from 0.40731
Epoch 48/50
592/592 [==============================] - 4s 7ms/step - loss: 

Testiranje modela

In [12]:
import glob
maxlen = 100
batch_size = 32
loss = 'categorical_crossentropy'
model_json='hb_model_orthogonal_experiment_norm.json'
weights='Model2_weights.hdf5'

def test(filename, model_json='hb_model_orthogonal_experiment_norm.json', 
         weights='Model1_weights.hdf5', 
         optimizer='adam', 
         loss='categorical_crossentropy'):
   
    #loads filename, transfrorms data using short-time Fourier transform, logscales the result and splits it into 129x129 squares
    X = dataimport.data_from_file(filename=str(filename)+".wav", width=129, height=256, max_frames=10)
    
    predictions = np.zeros(len(X))
    z = 0
    
    #Makes predictions for each 129x129 square
    for frame in X:
        predict_frame = np.zeros((1, 3, 129, 129))
        predict_frame[0] = frame
        predictions_all = model.predict_proba(predict_frame, batch_size=batch_size)
        predictions[z] = predictions_all[0][1]

        z += 1
    
    #Averages the results of per-frame predictions    
    average = np.average(predictions)
    average_prediction = round(average)
    		
    #Prints the result
  #  if int(average_prediction) == 0.0:
        #append file with -1
       # write_answer(filename=filename, result="-1")
        
      #  print('Result for '+filename+': '+'Normal (-1)')
        
   # else:
        #append file with 1
       # write_answer(filename=filename, result="1")    
        
     #   print('Result for '+filename+': '+'Abnormal (1)')        
    
    return int(average_prediction)






import os
os.chdir("C:\\Users\\Josip")

model = model_from_json(open(model_json).read())

model.load_weights(weights)

optimizer='adam'

loss='categorical_crossentropy'

model.compile(loss=loss, optimizer=optimizer)




abnormal1=[]
normal1=[]
os.chdir("C:\\Users\\Josip\\new_test\\abnormal\\")
for filename in glob.glob('*.wav'):
    abnormal1.append(test(filename[:-4], model_json='hb_model_orthogonal_experiment_norm.json', 
         weights='Model2_weights.hdf5', 
         optimizer='adam', 
         loss='categorical_crossentropy'))


os.chdir("C:\\Users\\Josip\\new_test\\normal\\")
for filename in glob.glob('*.wav'):
    normal1.append(test(filename[:-4], model_json='hb_model_orthogonal_experiment_norm.json', 
         weights='Model2_weights.hdf5', 
         optimizer='adam', 
         loss='categorical_crossentropy'))

print("Recall")
print(sum(abnormal1)/len(abnormal1))

print("Specificity")
print(1-sum(normal1)/len(normal1))

print("Mean score")
print( (sum(abnormal1)/len(abnormal1)+1-sum(normal1)/len(normal1))  /2)

Recall
0.8
Specificity
0.9285714285714286
Mean score
0.8642857142857143
